In [25]:
import jax.random as jrn
import jax.numpy as jnp
from bots import bots, run, mask


In [78]:
propose_bot = run.fuse(bots.propose_random, bots.propose_random, bots.propose_random)


def vote_bot_random(**_):
    return jnp.array(0.55)


vote_bot = run.fuse(vote_bot_random, vote_bot_random, vote_bot_random)

presi_bot = run.fuse(
    bots.discard_true,
    bots.discard_false,
    bots.discard_false,
)

chanc_bot = run.fuse(bots.discard_true, bots.discard_false, bots.discard_false)

shoot_bot = run.fuse(bots.shoot_random, bots.shoot_random, bots.shoot_random)

total = 5

run_func = run.closure(
    total,
    15,
    propose_bot=propose_bot,
    vote_bot=vote_bot,
    presi_bot=presi_bot,
    chanc_bot=chanc_bot,
    shoot_bot=shoot_bot,
)


In [77]:
params = {
    "propose": 0,
    "vote": 0,
    "presi": 0,
    "chanc": 0,
    "shoot": 0,
}

state = run_func(jrn.PRNGKey(0), params)  # type: ignore
state = mask.mask(state)

player = 4

for k, v in state.items():
    state[k] = v[player]


In [85]:
board = state["board"]
tracker = state["tracker"]
presi = state["presi"]
chanc = state["chanc"]

new_policies = board[:-1] - board[1:]
# print(new_policies)

enacted = tracker == 0
enacted &= presi != -1

fometer_enact = new_policies  # * elected[:, None]

fometer_enact = fometer_enact.argmax(-1)
fometer_enact = 2 * fometer_enact - 1
fometer_enact = fometer_enact * enacted[:-1]
fometer_enact, presi[:-1], chanc[:-1]

(Array([-1, -1,  0,  0,  0,  1,  0,  1, -1,  1, -1,  0,  0,  0], dtype=int32),
 Array([ 0,  4,  3,  2,  1,  0,  4,  3,  2,  1,  0, -1, -1, -1], dtype=int32),
 Array([ 3,  2,  3,  3,  3,  3,  1,  1,  4,  3,  2, -1, -1, -1], dtype=int32))

In [86]:
fometer_presi_enact = jnp.zeros([total])
fometer_presi_enact = fometer_presi_enact.at[presi[:-1]].add(fometer_enact)

fometer_chan_enact = jnp.zeros([total])
fometer_chan_enact = fometer_chan_enact.at[chanc[:-1]].add(fometer_enact)

fometer_presi_enact, fometer_chan_enact

(Array([-1.,  1., -1.,  1., -1.], dtype=float32),
 Array([ 0.,  1., -2.,  1., -1.], dtype=float32))

In [68]:
tracker, board  # [:-1]


(Array([0, 0, 3, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 Array([[5, 3],
        [4, 3],
        [3, 3],
        [2, 3],
        [2, 3],
        [2, 3],
        [2, 2],
        [2, 2],
        [2, 1],
        [1, 1],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]], dtype=int32))

In [69]:
state["presi"]


Array([ 0,  4,  3,  2,  1,  0,  4,  3,  2,  1,  0, -1, -1, -1, -1], dtype=int32)